In [1]:
import os
import sys
import pandas as pd
import torch
from transformers import AutoTokenizer, T5EncoderModel
import argparse
from torch.utils.data import DataLoader
import numpy as np
sys.path.append("../")
from utils import seed_everything, filter_out
from generation_utils import ReactionT5Dataset
from train import preprocess_df

/home/sagawa/miniconda3/envs/reactiont5/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("/data1/ReactionT5_neword/task_yield/output_test1_test/input_data.csv")
df

,REACTANT,REAGENT,PRODUCT,YIELD,CATALYST,SOLVENT,input
0,Cc1ccc(N)cc1.FC(F)(F)c1ccc(Cl)cc1,CC(C)c1cc(C(C)C)c(-c2ccccc2P(C2CCCCC2)(C2CCCCC...,Cc1ccc(Nc2ccc(C(F)(F)F)cc2)cc1,0.106578,,,REACTANT:Cc1ccc(N)cc1.FC(F)(F)c1ccc(Cl)cc1REAG...
1,Cc1ccc(N)cc1.FC(F)(F)c1ccc(Br)cc1,CC(C)c1cc(C(C)C)c(-c2ccccc2P(C2CCCCC2)(C2CCCCC...,Cc1ccc(Nc2ccc(C(F)(F)F)cc2)cc1,0.147479,,,REACTANT:Cc1ccc(N)cc1.FC(F)(F)c1ccc(Br)cc1REAG...
2,Cc1ccc(N)cc1.FC(F)(F)c1ccc(I)cc1,CC(C)c1cc(C(C)C)c(-c2ccccc2P(C2CCCCC2)(C2CCCCC...,Cc1ccc(Nc2ccc(C(F)(F)F)cc2)cc1,0.182787,,,REACTANT:Cc1ccc(N)cc1.FC(F)(F)c1ccc(I)cc1REAGE...
3,COc1ccc(Cl)cc1.Cc1ccc(N)cc1,CC(C)c1cc(C(C)C)c(-c2ccccc2P(C2CCCCC2)(C2CCCCC...,COc1ccc(Nc2ccc(C)cc2)cc1,0.024751,,,REACTANT:COc1ccc(Cl)cc1.Cc1ccc(N)cc1REAGENT:CC...
4,COc1ccc(Br)cc1.Cc1ccc(N)cc1,CC(C)c1cc(C(C)C)c(-c2ccccc2P(C2CCCCC2)(C2CCCCC...,COc1ccc(Nc2ccc(C)cc2)cc1,0.061191,,,REACTANT:COc1ccc(Br)cc1.Cc1ccc(N)cc1REAGENT:CC...
...,...,...,...,...,...,...,...
893,Brc1ccccn1.Cc1ccc(N)cc1,CN1CCCN2CCCN=C12.COC(=O)c1cc(-c2cccs2)on1.COc1...,Cc1ccc(Nc2ccccn2)cc1,0.574267,,,REACTANT:Brc1ccccn1.Cc1ccc(N)cc1REAGENT:CN1CCC...
894,Cc1ccc(N)cc1.Ic1ccccn1,CN1CCCN2CCCN=C12.COC(=O)c1cc(-c2cccs2)on1.COc1...,Cc1ccc(Nc2ccccn2)cc1,0.862332,,,REACTANT:Cc1ccc(N)cc1.Ic1ccccn1REAGENT:CN1CCCN...
895,Cc1ccc(N)cc1.Clc1cccnc1,CN1CCCN2CCCN=C12.COC(=O)c1cc(-c2cccs2)on1.COc1...,Cc1ccc(Nc2cccnc2)cc1,0.014401,,,REACTANT:Cc1ccc(N)cc1.Clc1cccnc1REAGENT:CN1CCC...
896,Brc1cccnc1.Cc1ccc(N)cc1,CN1CCCN2CCCN=C12.COC(=O)c1cc(-c2cccs2)on1.COc1...,Cc1ccc(Nc2cccnc2)cc1,0.435384,,,REACTANT:Brc1cccnc1.Cc1ccc(N)cc1REAGENT:CN1CCC...


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/data1/ReactionT5_neword/task_yield")
tokenizer

In [6]:
df["len"] = df["input"].apply(lambda x: len(tokenizer(x)["input_ids"]))
df["len"].describe()

count    898.000000
mean     200.923163
std       21.240663
min      161.000000
25%      184.000000
50%      201.000000
75%      215.000000
max      260.000000
Name: len, dtype: float64

In [7]:
df = pd.read_csv("/data1/ReactionT5_neword/task_yield/output_inputmaxlength150/output_ord_test1/input_data.csv")
df["len"] = df["input"].apply(lambda x: len(tokenizer(x)["input_ids"]))
df["len"].describe()

count    545276.000000
mean        112.463938
std          51.235974
min          10.000000
25%          77.000000
50%         102.000000
75%         136.000000
max        1128.000000
Name: len, dtype: float64

In [8]:
df1 = pd.read_csv("/data1/ReactionT5_neword/task_yield/output_ord_test1/input_data.csv")
df2  = pd.read_csv("/data1/ReactionT5_neword/task_yield/output_ord_test2/input_data.csv")
df1 == df2

,id,CATALYST,REACTANT,REAGENT,SOLVENT,INTERNAL_STANDARD,NoData,PRODUCT,YIELD,TEMP,input
0,True,True,True,True,True,False,False,True,True,False,True
1,True,True,True,True,True,False,False,True,True,False,True
2,True,True,True,True,True,False,False,True,True,False,True
3,True,True,True,True,True,False,False,True,True,True,True
4,True,True,True,True,True,False,False,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...
545271,True,True,True,True,True,False,False,True,True,False,True
545272,True,True,True,True,True,False,False,True,True,False,True
545273,True,True,True,True,True,False,False,True,True,False,True
545274,True,True,True,True,True,False,False,True,True,False,True


In [11]:
(df1["id"] == df2["id"]).all()

True

In [10]:
df2

,id,CATALYST,REACTANT,REAGENT,SOLVENT,INTERNAL_STANDARD,NoData,PRODUCT,YIELD,TEMP,input
0,ord-b45f2eddde7844199a0384f3f9ed9b3e,,C(=NC1CCCCC1)=NC1CCCCC1.CC(C)(C)OC(=O)CO/N=C(\...,,CC(=O)N(C)C.CC(=O)N(C)C.O,NaN,NaN,CC(C)(C)OC(=O)CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(...,0.83,NaN,REACTANT:C(=NC1CCCCC1)=NC1CCCCC1.CC(C)(C)OC(=O...
1,ord-c7e5a1f8ae464c78babc02d43cb78f73,,CC(C)OC(=O)N=NC(=O)OC(C)C.CCCCC[C@H](O)c1ccc(C...,,C1CCOC1,NaN,NaN,CCCCC[C@@H](OC(=O)c1ccc([N+](=O)[O-])cc1)c1ccc...,0.63,NaN,REACTANT:CC(C)OC(=O)N=NC(=O)OC(C)C.CCCCC[C@H](...
2,ord-6076322030904576890bf19a81d0c79c,,C[O-].Cl.N#CCc1coc2c(Cl)cccc12.NO.[Na+],,CO.CO,NaN,NaN,NC(Cc1coc2c(Cl)cccc12)=NO,0.73,NaN,REACTANT:C[O-].Cl.N#CCc1coc2c(Cl)cccc12.NO.[Na...
3,ord-25f6d0e6175d4f8fa4ab8b83f527d95f,,CCCc1cc(C(N)=S)ccn1.O=C(CBr)c1ccc(Br)cc1,,CCO,NaN,NaN,CCCc1cc(-c2nc(-c3ccc(Br)cc3)cs2)ccn1,0.79,70.0,REACTANT:CCCc1cc(C(N)=S)ccn1.O=C(CBr)c1ccc(Br)...
4,ord-8784a4a9d51847238a9cfb01b2e59f79,,CC(C)(C)c1ccc(C(=O)Cl)cc1.COC(=O)c1ccc(N)c(N)c...,,,NaN,NaN,COC(=O)c1ccc(N)c(NC(=O)c2ccc(C(C)(C)C)cc2)c1,0.44,NaN,REACTANT:CC(C)(C)c1ccc(C(=O)Cl)cc1.COC(=O)c1cc...
...,...,...,...,...,...,...,...,...,...,...,...
545271,ord-97ee57f9f7de4c5ebfb4267d8e649131,,BrCc1ccccc1.CCCI.CCOC(=O)c1sc(N2CCNC2=O)nc1C,,,NaN,NaN,CCCN1CCN(c2nc(C)c(C(=O)OCC)s2)C1=O,0.60,NaN,REACTANT:BrCc1ccccc1.CCCI.CCOC(=O)c1sc(N2CCNC2...
545272,ord-06550951fec441f6bdc52d72fb380b01,CC1(C)CCCC(C)(C)N1[O],CC(C)(C)OC(=O)N1CCC[C@@H]([C@@](O)(CCCCO)c2ccc...,CC1(C)CCCC(C)(C)N1[O],CC(C)=O.CC(C)O,NaN,NaN,CC(C)(C)OC(=O)N1CCC[C@@H]([C@]2(c3cccc(Cl)c3)C...,0.81,NaN,REACTANT:CC(C)(C)OC(=O)N1CCC[C@@H]([C@@](O)(CC...
545273,ord-579d124186cd4fb0ad4bc1329b2f5fe6,Cl[Pd]Cl.[Cu]I.c1ccc(P(c2ccccc2)c2ccccc2)cc1.c...,C#CCO.COc1cc(I)cc2c(O)c(C(=O)NCc3ccc(Cl)cc3)cn...,Cl[Pd]Cl.[Cu]I.c1ccc(P(c2ccccc2)c2ccccc2)cc1.c...,CCNCC,NaN,NaN,COc1cc(C#CCO)cc2c(O)c(C(=O)NCc3ccc(Cl)cc3)cnc12,0.73,NaN,REACTANT:C#CCO.COc1cc(I)cc2c(O)c(C(=O)NCc3ccc(...
545274,ord-09ae3e6c4191438da6d726e930a280a7,,C/C(=C\CC(C#N)c1cccc(CO[Si](c2ccccc2)(c2ccccc2...,,,NaN,NaN,C/C(=C\CC(C=O)c1cccc(CO[Si](c2ccccc2)(c2ccccc2...,0.97,NaN,REACTANT:C/C(=C\CC(C#N)c1cccc(CO[Si](c2ccccc2)...
